# 경구약 안전 복용 서비스(Medicine Safe Dose Service)

## 필요한 함수 정의

In [ ]:
# 인공지능 학습과정에서 필요한 함수들을 정의한다.
import pandas as pd # -> DataFrame 을 생성하기 위한 함수
import json # -> 데이터 셋에 들어 있는 json 파일을 읽기 위한 함수
import numpy as np # -> DataFrame 을 생성하는데 쓰이는 함수로, numpy 로 배열을 만든 뒤 DataFrame 에 추가하기 위한 함수
import sys # ->
from PIL import Image, ImageDraw #->  파일의 이미지를 불러오기 위한 함수
import json 
import matplotlib.pyplot as plt
import torch.optim as optim
from math import *
import shutil
import copy
from torch.utils.checkpoint import checkpoint 
import time
from log import *
from torch.utils.data import Dataset
import xml.etree.ElementTree as et
from torchvision import datasets,transforms,utils
from torch.utils.data import DataLoader
import zipfile

import torch
import torch.nn as nn

from torchvision.models import * 
from torch.optim import lr_scheduler

import os
# -> cuda(gpu)를 사용할 때 에러가 날 경우 에러를 자세히 보기 위한 코드
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import gc
# -> 파이썬이 처음 시작할 때 생성하는 많은 객체들을 효율적으로 관리하기 위한 코드
gc.collect()
torch.cuda.empty_cache()


C:\Users\white\.cache\torch\hub\checkpoints
사전학습된 모델들이 있는 위치

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

## 학습을 위한 파라미터 세팅

In [ ]:
# 학습을 진행할 때에는 이 코드만 건들면 된다.

epoch=10
file_name="test1"
if not os.path.isdir("./result"):
    os.mkdir("./result")
Path="./result/"+file_name+"/"
log_path=Path+"Process_log.txt"
batch_size=256
# lr 0.001 batch size 256

lr=0.02
step_size=10
gamma=0.1


if os.path.isdir(Path):
    print("이미 폴더가 존재합니다.")
    sys.exit()
os.mkdir(Path)

print("log_path :",log_path)
TXT=log(log_path)
TXT.open()

## 1-2. DATA -> Augmentation -> Train : Test = 7 : 3

In [ ]:
def Data_Pretreatment(path="./166.약품식별 인공지능 개발을 위한 경구약제 이미지 데이터/01.데이터/1.Training/",
                      category=["object_name","RHF","RVF","RG","path"],
                      img_move_path="./DATA/",
                      img_move=0,
                      data_augmentation={"RHF":1,"RVF":1,"RG":1}):
    
    if os.path.isfile("./dataframe.csv"):
        DATAFRAME=pd.read_csv("./dataframe.csv")
        yshape=DATAFRAME.shape[0]
        object_type=[i for i in range(DATAFRAME["object_name"].iloc[yshape-1]+1)]
        return pd.read_csv("./dataframe.csv"),object_type


    label_path=path+"라벨링데이터/"
    image_path=path+"원천데이터/"
    if img_move==1:
        if not os.path.isdir("./DATA"):
            os.mkdir("./DATA")

    dataframe=np.array([])
    dataframe = np.empty((0,len(category)), int)
    
    img_draw_stack=0
    object_type=[]
    object_type_idx=0


    for big_category in os.listdir(image_path):
          
        big_category_path=image_path+big_category+"/"

        for mid_category in os.listdir(big_category_path):

            if len(mid_category.split(".zip"))>=2:continue
            mid_category_path=big_category_path+mid_category+"/"
            
            for object_name in os.listdir(mid_category_path):

                #print("object name :",object_name)

                object_path=mid_category_path+object_name+"/"
                #print(os.listdir(object_path))

                for file_name in os.listdir(object_path):

                    img2json_path=file_name.split(".")[0]+".json"
                    mid_category_replace=mid_category.replace("S","L")
                    object_label_path=label_path+big_category+"/"+mid_category_replace+"/"+object_name+"_json"+"/"+img2json_path
                    with open (object_label_path, "r",encoding='UTF-8') as f:
                        data = json.load(f)

                    NAME=data['images'][0]['dl_name'].replace("/","-")
                    object_file_path=img_move_path+NAME+"/"
   
                    if not NAME in object_type:
                        object_type.append(NAME)
                        object_type_idx+=1


                    Size=[data['images'][0]['width'],data['images'][0]['height']]
                    BOX=data['annotations'][0]['bbox']

                    if img_move==1:
                        img_size=(480,480)
                        img = Image.open(object_path+file_name).convert('RGB')
                        Size=img.size
                        if not os.path.isdir(object_file_path):
                            os.mkdir(object_file_path)
                        center=[(2*BOX[0]+BOX[2])/2,(2*BOX[1]+BOX[3])/2]
                        img=img.crop((center[0]-240,center[1]-240,center[0]+240,center[1]+240))
                        img=img.resize((240,240))
                        img.save(object_file_path+file_name,"PNG")
                        #shutil.copyfile(object_path+file_name, object_file_path+file_name)
                    dataframe=np.append(dataframe,np.array([[object_type_idx-1,0,0,0,object_file_path+file_name]]),axis=0)
                    if data_augmentation["RHF"]:
                        dataframe=np.append(dataframe,np.array([[object_type_idx-1,1,0,0,object_file_path+file_name]]),axis=0)
                    if data_augmentation["RVF"]:
                        dataframe=np.append(dataframe,np.array([[object_type_idx-1,0,1,0,object_file_path+file_name]]),axis=0)
                    if data_augmentation["RG"]:
                        dataframe=np.append(dataframe,np.array([[object_type_idx-1,0,0,1,object_file_path+file_name]]),axis=0)
                    #print(NAME,BOX)

    DATAFRAME=pd.DataFrame(data=dataframe,columns=category)

    return DATAFRAME,object_type

    

In [ ]:
def collcet_object_type(path="./datafile/"):

    img_path=path

    object_type=[]
    object_type_idx=0
    

    for object_name in os.listdir(img_path):

        if not object_name in object_type:
            object_type.append(object_name)
            object_type_idx+=1

    return [i for i in range(object_type_idx)]

    

In [ ]:
def DataFrame_dividing(DataFrame,category=["object_name","path"]):

    train=np.array([])
    train = np.empty((0,len(category)), int)
    test=np.array([])
    test = np.empty((0,len(category)), int)

    #print(DataFrame)
    

    class_stack=[]
    last_obj=-1

    #print(DataFrame.shape)
    #print(DATAFRAME)

    for i in range(DataFrame.shape[0]):

        Ip=[]
        for g in range(len(category)):Ip.append(DataFrame.iloc[i][category[g]])
        if last_obj==-1:
            last_obj=Ip[0]

        #print(last_obj,Ip[0])

        if last_obj!=Ip[0]:
            #print("##")
            T=int(len(class_stack)/10 * 7)
            #print("TT",T)
            for g in range(T):
                train=np.append(train,np.array([class_stack[g]]),axis=0)
                #print(train,np.array([class_stack[g]]),class_stack[g])
                #np.append(dataframe,np.array([[object_type_idx-1, BOX[0], BOX[1], BOX[0]+BOX[2], BOX[1]+BOX[3], object_file_path+file_name]]),axis=0)
            for g in range(T,len(class_stack)):
                test=np.append(test,np.array([class_stack[g]]),axis=0)

            class_stack=[]
            last_obj=Ip[0]
        class_stack.append(Ip)

    if class_stack!=[]:
        T=int(len(class_stack)/10 * 7)
        for g in range(T):
            train=np.append(train,np.array([class_stack[g]]),axis=0)
            #np.append(dataframe,np.array([[object_type_idx-1, BOX[0], BOX[1], BOX[0]+BOX[2], BOX[1]+BOX[3], object_file_path+file_name]]),axis=0)
        for g in range(T,len(class_stack)):
            test=np.append(test,np.array([class_stack[g]]),axis=0)
        
    train=pd.DataFrame(data=train,columns=category)
    test=pd.DataFrame(data=test,columns=category)

    return train,test



    

In [ ]:
# rotation 필요, 단일 데이터 라벨링 이기 때문에 다중 데이터 같은 경우 bounding box 에서 에러 날 수 있으니 확인
#image folder produce data file
def if_pd_datafile(path="./166.약품식별 인공지능 개발을 위한 경구약제 이미지 데이터/01.데이터/1.Training/",
                      category=["object_name","RHF","RVF","RG","path"],
                      img_move_path="./DATA/",
                      data_augmentation={"RHF":1,"RVF":1,"RG":1}):
    

    label_path=path+"라벨링데이터/"
    image_path=path+"원천데이터/"
    if not os.path.isdir("./DATA"):
         os.mkdir("./DATA")
    if not os.path.isdir("./DATA/train"):
         os.mkdir("./DATA/train")
    if not os.path.isdir("./DATA/test"):
         os.mkdir("./DATA/test")

    class_stack=[]
    
    object_type=[]
    object_type_idx=0


    for big_category in os.listdir(image_path):
          
        big_category_path=image_path+big_category+"/"

        for mid_category in os.listdir(big_category_path):

            if len(mid_category.split(".zip"))>=2:continue
            mid_category_path=big_category_path+mid_category+"/"
            
            for object_name in os.listdir(mid_category_path):

                #print("object name :",object_name)

                object_path=mid_category_path+object_name+"/"
                #print(os.listdir(object_path))

                for file_name in os.listdir(object_path):

                    img2json_path=file_name.split(".")[0]+".json"
                    mid_category_replace=mid_category.replace("S","L")
                    object_label_path=label_path+big_category+"/"+mid_category_replace+"/"+object_name+"_json"+"/"+img2json_path
                    with open (object_label_path, "r",encoding='UTF-8') as f:
                        data = json.load(f)

                    NAME=data['images'][0]['dl_name'].replace("/","-")
        
                    if not NAME in object_type:
                        if object_type!=[]:
                            T=int(len(class_stack)/10 * 7)
                            #train
                            for g in range(T):
                                DATA=class_stack[g]
                                BOX=DATA[4]

                                img = Image.open(DATA[3]).convert('RGB')
                                object_file_path=img_move_path+"train/"+DATA[2]+"/"
                                if not os.path.isdir(object_file_path):
                                    os.mkdir(object_file_path)

                                center=[(2*BOX[0]+BOX[2])/2,(2*BOX[1]+BOX[3])/2]
                                #img=img.crop((center[0]-240,center[1]-240,center[0]+240,center[1]+240))
                                img=img.crop((BOX[0],BOX[1],BOX[0]+BOX[2],BOX[1]+BOX[3]))
                                #print(BOX)
                                #print(img.size)
                                img=img.resize((240,240))

                                if DATA[1]["RHF"]:
                                    hflipper = transforms.RandomHorizontalFlip(p=0.5)
                                    img=hflipper(img)

                                elif DATA[1]["RVF"]:
                                    vflipper = transforms.RandomVerticalFlip(p=0.5)
                                    img=vflipper(img)

                                elif DATA[1]["RG"]:
                                    scaler = transforms.RandomGrayscale(p=0.75)
                                    img=scaler(img)

                                img.save(object_file_path+DATA[5],"PNG")


                            #test
                            for g in range(T,len(class_stack)):
                                DATA=class_stack[g]
                                BOX=DATA[4]

                                img = Image.open(DATA[3]).convert('RGB')
                                object_file_path=img_move_path+"test/"+DATA[2]+"/"
                                if not os.path.isdir(object_file_path):
                                    os.mkdir(object_file_path)

                                center=[(2*BOX[0]+BOX[2])/2,(2*BOX[1]+BOX[3])/2]
                                img=img.crop((BOX[0],BOX[1],BOX[0]+BOX[2],BOX[1]+BOX[3]))
                                img=img.resize((240,240))

                                if DATA[1]["RHF"]:
                                    hflipper = transforms.RandomHorizontalFlip(p=0.5)
                                    img=hflipper(img)

                                elif DATA[1]["RVF"]:
                                    vflipper = transforms.RandomVerticalFlip(p=0.5)
                                    img=vflipper(img)

                                elif DATA[1]["RG"]:
                                    scaler = transforms.RandomGrayscale(p=0.75)
                                    img=scaler(img)

                                img.save(object_file_path+DATA[5],"PNG")

                        object_type.append(NAME)
                        class_stack=[]
                        object_type_idx+=1

                    
                    class_stack.append([object_type_idx,data_augmentation,NAME,object_path+file_name,data['annotations'][0]['bbox'],file_name])
                    
                
                    #print(NAME,BOX)
    if class_stack!=[]:
        T=int(len(class_stack)/10 * 7)
        #train
        for g in range(T):
            DATA=class_stack[g]
            BOX=DATA[4]

            img = Image.open(DATA[3]).convert('RGB')
            object_file_path=img_move_path+"train/"+DATA[2]+"/"
            if not os.path.isdir(object_file_path):
                os.mkdir(object_file_path)

            center=[(2*BOX[0]+BOX[2])/2,(2*BOX[1]+BOX[3])/2]
            img=img.crop((BOX[0],BOX[1],BOX[0]+BOX[2],BOX[1]+BOX[3]))

            img=img.resize((240,240))
            if DATA[1]["RHF"]:
                hflipper = transforms.RandomHorizontalFlip(p=0.5)
                img=hflipper(img)

            elif DATA[1]["RVF"]:
                vflipper = transforms.RandomVerticalFlip(p=0.5)
                img=vflipper(img)

            elif DATA[1]["RG"]:
                scaler = transforms.RandomGrayscale(p=0.75)
                img=scaler(img)

            img.save(object_file_path+DATA[5],"PNG")


        #test
        for g in range(T,len(class_stack)):
            DATA=class_stack[g]
            BOX=DATA[4]

            img = Image.open(DATA[3]).convert('RGB')
            object_file_path=img_move_path+"test/"+DATA[2]+"/"
            if not os.path.isdir(object_file_path):
                os.mkdir(object_file_path)

            center=[(2*BOX[0]+BOX[2])/2,(2*BOX[1]+BOX[3])/2]
            img=img.crop((BOX[0],BOX[1],BOX[0]+BOX[2],BOX[1]+BOX[3]))
            img=img.resize((240,240))

            if DATA[1]["RHF"]:
                hflipper = transforms.RandomHorizontalFlip(p=0.5)
                img=hflipper(img)

            elif DATA[1]["RVF"]:
                vflipper = transforms.RandomVerticalFlip(p=0.5)
                img=vflipper(img)

            elif DATA[1]["RG"]:
                scaler = transforms.RandomGrayscale(p=0.75)
                img=scaler(img)

            img.save(object_file_path+DATA[5],"PNG")
            

    return object_type

    

## 1-3. Train & Test model

In [ ]:
def TransForm(size=256):
    train_transform = transforms.Compose([
        #transforms.Resize(size),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        #transforms.RandomVerticalFlip(),
        #transforms.RandomGrayscale(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) 
    ])

    test_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) 
    ])
    return train_transform, test_transform

In [ ]:
class CustomDataset(Dataset):
    def __init__(self,labels,transforms):
        self.labels=labels
        #print(self.labels)
        self.transforms=transforms

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        img_path=self.labels["path"].iloc[idx]
        label=self.labels["object_name"].iloc[idx]
        #print(idx,self.labels.iloc[idx])

        img=Image.open(img_path)

        if self.labels["RHF"].iloc[idx]:
            hflipper = transforms.RandomHorizontalFlip(p=0.5)
            img=hflipper(img)

        elif self.labels["RVF"].iloc[idx]:
            vflipper = transforms.RandomVerticalFlip(p=0.5)
            img=vflipper(img)

        elif self.labels["RG"].iloc[idx]:
            scaler = transforms.RandomGrayscale(p=0.75)
            img=scaler(img)

        if self.transforms:
            img=self.transforms(img)

       # OUTPUT={"iamge":img,"label":label}

        return img,label

In [ ]:
def produce_Dataset(train_dataset,test_dataset,train_transform,test_transform,batch_size=64):

  #[train,train_transform]
  #print("train")
  train_dataset = CustomDataset(train_dataset,train_transform)
  #print("test")
  test_dataset = CustomDataset(test_dataset,test_transform)

  train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

  test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

  return {"dataset":train_dataset,"dataloader":train_dataloader},{"dataset":test_dataset,"dataloader":test_dataloader}


In [ ]:
def IF_produce_Dataset(train_transform,test_transform,batch_size=64):

  train_dataset = datasets.ImageFolder(os.path.join("./DATA/train"),train_transform)

  test_dataset = datasets.ImageFolder(os.path.join("./DATA/test"),test_transform)

  train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

  test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

  return {"dataset":train_dataset,"dataloader":train_dataloader},{"dataset":test_dataset,"dataloader":test_dataloader}


In [ ]:
def txt_print(input):
    TXT.print_log(input)
    

In [ ]:
def train_test_model(TRAIN,TEST,EPOCH,PATH):


    #sys.stdout=open(PATH+"train_result.txt","w")
    since = time.time()
    train_best_acc=0.0
    train_best_loss=inf
    train_idx=0
    test_best_acc=0.0
    test_best_loss=inf
    test_idx=0
    

    for epoch in range(EPOCH):
        TXT.print_log(f'Epoch {epoch}/{EPOCH - 1}')
        TXT.print_log('-' * 10)

        ###############################################
        #--------------------train--------------------#
        ###############################################

        TRAIN["model"].train()  # 모델을 학습 모드로 설정


        running_loss = 0.0
        running_corrects = 0.0 #[0]*(len(MPI.object_stack)+2)
        #print(next(iter(TRAIN["learning_dataloader"])))

        # 데이터를 반복
        for i,batch in enumerate(TRAIN["learning_dataloader"]):
            #print(f"i : {i}, batch : {batch}.")

            
            inputs,labels=batch

            inputs = inputs.to(device).cuda()
            #print(labels)
            #labels=torch.tensor(labels)
            labels = torch.LongTensor.clone(labels).to(device)
            labels = labels.to(device)

            outputs = TRAIN["model"](inputs)

            _, preds = torch.max(outputs, 1)

            loss = TRAIN["criterion"](outputs, labels)


            TRAIN["optimizer"].zero_grad()
            #loss.requires_grad_(True)
            loss.backward()
            #"""
            
            TRAIN["optimizer"].step()
            
            # 통계
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        TRAIN["scheduler"].step()

        epoch_loss = running_loss / TRAIN["learning_dataset_size"]
        epoch_acc = running_corrects.double() / TRAIN["learning_dataset_size"]
        
        TXT.print_log(f'train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        if epoch_acc > train_best_acc:
            train_best_acc = epoch_acc
            train_idx=epoch+1
            train_best_loss=epoch_loss

        torch.save(TRAIN["model"].state_dict(), PATH+'checkpoint_epoch_'+str(epoch+1)+'.pth')

        ##############################################
        #--------------------test--------------------#
        ##############################################

        TEST["model"].load_state_dict(torch.load(PATH+'checkpoint_epoch_'+str(epoch+1)+'.pth'))

        TEST["model"].eval()  # 모델을 학습 모드로 설정

        running_loss = 0.0
        running_corrects = 0.0 #[0]*(len(MPI.object_stack)+2)

        # 데이터를 반복
        for i,batch in enumerate(TEST["learning_dataloader"]):

            inputs,labels=batch
            inputs = inputs.to(device)
            #labels=torch.tensor(labels)
            labels = torch.LongTensor.clone(labels).to(device)

            outputs = TEST["model"](inputs)
            _, preds = torch.max(outputs, 1)
            loss = TEST["criterion"](outputs, labels)

            # 통계
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / TEST["learning_dataset_size"]
        epoch_acc = running_corrects.double() / TEST["learning_dataset_size"]
        
        TXT.print_log(f'test Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        if epoch_acc > test_best_acc:
            test_best_acc = epoch_acc
            test_idx=epoch+1
            test_best_loss=epoch_loss

        #TXT.close()
        #TXT.open()

    time_elapsed = time.time() - since

    TXT.print_log('')
    TXT.print_log(f'Training & Testing complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    TXT.print_log(f'Best train Acc: {train_best_acc}')
    TXT.print_log(f'Best train Acc\'s loss : {train_best_loss}')
    TXT.print_log(f'Best train Acc\'s epoch : {train_idx}')
    TXT.print_log("")
    TXT.print_log(f'Best test Acc: {test_best_acc}')
    TXT.print_log(f'Best test Acc\'s loss : {test_best_loss}')
    TXT.print_log(f'Best test Acc\'s epoch : {test_idx}')


In [ ]:
TXT.print_log("Hyperparameter setting")
TXT.print_log("----------------------")

#---------------------Dataset setting---------------------#

category=["object_name","RHF","RVF","RG","path"]
#object_type=if_pd_datafile(category=category)
object_type=[0]*5
train_transform,test_transform=TransForm(240)
"""
DATAFRAME,object_type=Data_Pretreatment(category=category,img_move=0)
train,test=DataFrame_dividing(DATAFRAME,category)

#print("test")
#print(test)

Train,Test=produce_Dataset(train,test,train_transform,test_transform,batch_size)
"""
Train,Test=IF_produce_Dataset(train_transform,test_transform,batch_size)

learning_dataset={"train":Train["dataset"],"test":Test["dataset"]}
learning_dataloader={"train":Train["dataloader"],"test":Test["dataloader"]}
learning_dataset_size={"train":len(Train["dataset"]),"test":len(Test["dataset"])}

# ---------------Dataset setting---------------------#


#---------------------model---------------------#
model_conv = resnet152(weights="ResNet152_Weights.IMAGENET1K_V2")#(weights="ResNet152_Weights.IMAGENET1K_V1")
TXT.print_log(str(f"model is resnet18. weights is ResNet152_Weights.IMAGENET1K_V1"))
for param in model_conv.parameters():
    param.requires_grad = False

#resnet
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, len(object_type))

#densenet
#num_ftrs = model_conv.classifier.in_features
#model_conv.classifier = nn.Linear(num_ftrs, len(object_type))

#efficientnet
#num_ftrs = model_conv.classifier[1].in_features
#model_conv.classifier[1] = nn.Linear(num_ftrs, len(object_type))

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()
TXT.print_log(str("Loss Function is CrossEntropyLoss"))
optimizer_conv = optim.Adam(model_conv.fc.parameters(),lr=lr)
TXT.print_log(str(f"optimizer is Adam. lr is {lr}."))
TXT.print_log(str(f'batch size is {batch_size}.'))
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=step_size, gamma=gamma)
TXT.print_log(str(f"lr_scheduler is StepLR."))
TXT.print_log(str(f'step size is {step_size}.'))
TXT.print_log(str(f'gamma is {gamma}.'))

#---------------------model---------------------#
TXT.print_log("---------------------------")
#TXT.close()

In [ ]:
#train_model(model, criterion, optimizer, scheduler, learning_dataloader,learning_dataset_size,EPOCH,PATH)
#TXT.open()
TXT.print_log("train & test log")
TXT.print_log("---------")
TRAIN={"model":model_conv,
       "criterion":criterion,
       "optimizer":optimizer_conv,
       "scheduler":exp_lr_scheduler,
        "learning_dataloader":learning_dataloader["train"],
        "learning_dataset_size":learning_dataset_size["train"],
}
TEST={"model":model_conv,
       "criterion":criterion,
       "optimizer":optimizer_conv,
       "scheduler":exp_lr_scheduler,
        "learning_dataloader":learning_dataloader["test"],
        "learning_dataset_size":learning_dataset_size["test"],
}

train_test_model(TRAIN,TEST,epoch,Path)

TXT.print_log("---------------------------")
TXT.close()



In [ ]:
#DATAFRAME.to_csv('dataframe.csv')